# Wallaroo `upload_model` API Example

In order to call the `upload_model` endpoint, you need to have the following things ready:

- API Token
- Input/Output Arrow Schemas, encoded in base64
- `upload_model` usual arguments (i.e. model name, requirements, framework etc.)

### `upload_model` Example Call

```bash
curl --progress-bar -X POST \
  -H "Content-Type: multipart/form-data" \
  -H "Authorization: Bearer <token here>" \
  -F 'metadata={ "name": "gpt4all", "visibility": "private", "workspace_id": 6, "conversion": {"framework": "hugging-face-text-generation", "python_version": "3.8", "requirements": []}, "input_schema": <base64 input schema here>, "output_schema": <base64 output schema here>};type=application/json' \
  -F "file=@model-auto-conversion_hugging-face_LLM_gpt4all-groovy-hf-pipeline.zip;type=application/octet-stream" \
  https://autoscale-uat-ee.wallaroo.dev/v1/api/models/upload_and_convert | cat
```

### Generating Input/Output Encoded Schemas

```python
base64.b64encode(
                bytes(input_schema.serialize())
            ).decode("utf8")

base64.b64encode(
                bytes(output_schema.serialize())
            ).decode("utf8")
```

## Python Example

In order to show a concrete example of a request made in Python, we will show a HuggingFace Text Classification model uploaded with an API call.

### Imports

In [ ]:
import json
import os
import requests
import base64

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd

wl = wallaroo.Client(auth_type="sso", interactive=True)

### Get framework

In [ ]:
[e.value for e in Framework]

In [ ]:
Framework.HUGGING_FACE_TEXT_CLASSIFICATION

### Configure PyArrow Schemas

In [ ]:
input_schema = pa.schema([
    pa.field('inputs', pa.string()), # required
    pa.field('top_k', pa.int64()), # optional
])

output_schema = pa.schema([
    pa.field('label', pa.list_(pa.string(), list_size=2)), # list with a number of items same as top_k
    pa.field('score', pa.list_(pa.float64(), list_size=2)), # list with a number of items same as top_k
])

In [ ]:
encoded_input_schema = base64.b64encode(
                bytes(input_schema.serialize())
            ).decode("utf8")
encoded_input_schema

In [ ]:
encoded_output_schema = base64.b64encode(
                bytes(output_schema.serialize())
            ).decode("utf8")
encoded_output_schema

### Build the request

In [ ]:
API_TOKEN = "token"

In [ ]:
model_name = "gpt4all"
workspace_id = 6
framework = "hugging-face-text-generation"
model_path = "model-auto-conversion_hugging-face_LLM_gpt4all-groovy-hf-pipeline.zip"

In [ ]:
metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace_id,
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

metadata_str = f"{str(metadata)};type=application/json"
metadata_str

In [ ]:

headers = {
    'Content-Type': 'multipart/form-data',
    'Authorization': f'Bearer {API_TOKEN}',
}

files = {
    'metadata': metadata_str,
    'file': open(f'{model_path};type=application/octet-stream', 'rb'),
}

response = requests.post('https://autoscale-uat-ee.wallaroo.dev/v1/api/models/upload_and_convert', headers=headers, files=files)